In [ ]:
import sys, os; sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__) if '__file__' in globals() else os.getcwd(), '..')))
#import os; os.chdir(os.path.dirname(os.getcwd()))
from utils.model_loader import get_model_fits
import numpy as np
import pandas as pd
import re
from sklearn.metrics import mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
data_dir = f"datasets/friedman"
results_dir_tanh = "results/regression/single_layer/tanh/friedman"
model_names_tanh = ["Gaussian tanh", "Regularized Horseshoe tanh", "Dirichlet Horseshoe tanh", "Dirichlet Student T tanh", "Beta Horseshoe tanh", "Beta Student T tanh"]
model_names_tanh_nodewise = ["Dirichlet Horseshoe tanh nodewise", "Dirichlet Student T tanh nodewise", "Beta Horseshoe tanh nodewise", "Beta Student T tanh nodewise"]

tanh_fits = {}
tanh_fits_nodewise = {}

files = sorted(f for f in os.listdir(data_dir) if f.endswith(".npz"))
for fname in files:
    base_config_name = fname.replace(".npz", "")  # e.g., "GAM_N100_p8_sigma1.00_seed1"
    full_config_path = f"{base_config_name}"  # → "type_1/GAM_N100_p8_sigma1.00_seed1"
    
    tanh_fit = get_model_fits(
        config=full_config_path,
        results_dir=results_dir_tanh,
        models=model_names_tanh,
        include_prior=False,
    )
    
    tanh_fits[base_config_name] = tanh_fit  # use clean key
    
    tanh_fit_nodewise = get_model_fits(
        config=full_config_path,
        results_dir=results_dir_tanh,
        models=model_names_tanh_nodewise,
        include_prior=False,
    )
    
    tanh_fits_nodewise[base_config_name] = tanh_fit_nodewise  # use clean key
    


In [ ]:
data_dir_correlated = f"datasets/friedman_correlated"
results_dir_tanh_correlated = "results/regression/single_layer/tanh/friedman_correlated"

model_names_tanh_correlated = ["Gaussian tanh", "Regularized Horseshoe tanh", "Dirichlet Horseshoe tanh", "Dirichlet Student T tanh", "Beta Horseshoe tanh", "Beta Student T tanh"]
model_names_tanh_correlated_nodewise = ["Dirichlet Horseshoe tanh nodewise", "Dirichlet Student T tanh nodewise", "Beta Horseshoe tanh nodewise", "Beta Student T tanh nodewise"]

tanh_fits_correlated = {}
tanh_fits_correlated_nodewise = {}

files = sorted(f for f in os.listdir(data_dir_correlated) if f.endswith(".npz"))
for fname in files:
    base_config_name = fname.replace(".npz", "")  # e.g., "GAM_N100_p8_sigma1.00_seed1"
    full_config_path = f"{base_config_name}"  # → "type_1/GAM_N100_p8_sigma1.00_seed1"
    tanh_fit_correlated = get_model_fits(
        config=full_config_path,
        results_dir=results_dir_tanh_correlated,
        models=model_names_tanh_correlated,
        include_prior=False,
    )
    
    tanh_fits_correlated[base_config_name] = tanh_fit_correlated  # use clean key
    
    tanh_fit_correlated_nodewise = get_model_fits(
        config=full_config_path,
        results_dir=results_dir_tanh_correlated,
        models=model_names_tanh_correlated_nodewise,
        include_prior=False,
    )
    
    tanh_fits_correlated_nodewise[base_config_name] = tanh_fit_correlated_nodewise  # use clean key
    


In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from utils.generate_data import sample_gaussian_copula_uniform

def generate_Friedman_data_v2(N=100, D=10, sigma=1.0, test_size=0.2, seed=42, standardize_y=True, return_scale=True):
    np.random.seed(seed)
    X = np.random.uniform(0, 1, size=(N, D))
    x0, x1, x2, x3, x4 = X[:, 0], X[:, 1], X[:, 2], X[:, 3], X[:, 4]

    y_clean = (
        10 * np.sin(np.pi * x0 * x1) +
        20 * (x2 - 0.5) ** 2 +
        10 * x3 +
        5.0 * x4
    )
    y = y_clean + np.random.normal(0, sigma, size=N)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)

    if not standardize_y:
        return (X_train, X_test, y_train, y_test) if not return_scale else (X_train, X_test, y_train, y_test, 0.0, 1.0)

    y_mean = y_train.mean()
    y_std = y_train.std() if y_train.std() > 0 else 1.0

    y_train_s = (y_train - y_mean) / y_std
    y_test_s = (y_test - y_mean) / y_std

    if return_scale:
        return X_train, X_test, y_train_s, y_test_s, y_mean, y_std
    return X_train, X_test, y_train_s, y_test_s

def generate_correlated_Friedman_data_v2(N=100, D=10, sigma=1.0, test_size=0.2, seed=42, standardize_y=True, return_scale=True):
    """
    Generate synthetic regression data for Bayesian neural network experiments.

    Parameters:
        N (int): Number of samples.
        D (int): Number of features.
        sigma (float): Noise level.
        test_size (float): Proportion for test split.
        seed (int): Random seed.
        standardize_y (bool): Whether to standardize the response variable.

    Returns:
        tuple: (X_train, X_test, y_train, y_test, y_mean, y_std) if standardize_y,
               else (X_train, X_test, y_train, y_test)
    """
    np.random.seed(seed)
    d = 10
    S_custom = np.eye(d)
    # Block 1 (vars 0..4): high Spearman, 0.7
    for i in range(0, 3):
        for j in range(i+1, 3):
            S_custom[i, j] = S_custom[j, i] = 0.8
    # Block 2 (vars 5..9): moderate Spearman, 0.4
    for i in range(5, 10):
        for j in range(i+1, 10):
            S_custom[i, j] = S_custom[j, i] = -0.5
    # Cross-block weaker, 0.15
    for i in range(0, 5):
        for j in range(5, 10):
            S_custom[i, j] = S_custom[j, i] = 0.15
    # A couple of bespoke pairs:
    S_custom[0, 9] = S_custom[9, 0] = 0.4
    S_custom[2, 7] = S_custom[7, 2] = 0.9  # very strong (will be projected if infeasible)
    S_custom[3, 4] = S_custom[4, 3] = -0.9  # very strong (will be projected if infeasible)
    S_custom[1, 6] = S_custom[6, 1] = -0.9  # very strong (will be projected if infeasible)

    U, _ = sample_gaussian_copula_uniform(n=10000, S=S_custom, random_state=123)
    #X = np.random.uniform(0, 1, size=(N, D))
    if N != U.shape[0]:
        idx = np.random.choice(U.shape[0], size=N, replace=False)
        X = U[idx, :]
    else:
        X = U

    x0, x1, x2, x3, x4 = X[:, 0], X[:, 1], X[:, 2], X[:, 3], X[:, 4]

    y_clean = (
        10 * np.sin(np.pi * x0 * x1) +
        20 * (x2 - 0.5) ** 2 +
        10 * x3 +
        5.0 * x4
    )

    y = y_clean + np.random.normal(0, sigma, size=N)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)

    if not standardize_y:
        return (X_train, X_test, y_train, y_test) if not return_scale else (X_train, X_test, y_train, y_test, 0.0, 1.0)

    y_mean = y_train.mean()
    y_std = y_train.std() if y_train.std() > 0 else 1.0

    y_train_s = (y_train - y_mean) / y_std
    y_test_s = (y_test - y_mean) / y_std

    if return_scale:
        return X_train, X_test, y_train_s, y_test_s, y_mean, y_std
    return X_train, X_test, y_train_s, y_test_s

def make_large_eval_set(
    generator_fn,
    N_train,
    D,
    sigma,
    seed,
    n_eval=5000,
    standardize_y=True
):
    """
    Returns X_eval, y_eval (standardized if standardize_y=True), plus y_mean,y_std
    defined from the training split.
    """
    N_total = N_train + n_eval

    X_tr, X_te, y_tr, y_te, y_mean, y_std = generator_fn(
        N=N_total, D=D, sigma=sigma, test_size=n_eval / N_total, seed=seed,
        standardize_y=standardize_y, return_scale=True
    )
    # Now X_te has approx n_eval points (exact given test_size construction).
    return X_te, np.asarray(y_te).squeeze(), y_mean, y_std


In [13]:
from utils.sparsity import forward_pass_relu, forward_pass_tanh, local_prune_weights

def compute_sparse_rmse_results(seeds, models, all_fits, get_N_sigma, forward_pass, folder,
                         sparsity=0.0, prune_fn=None):
    results = []
    posterior_means = []
    
    def choose_gen(folder):
        return generate_correlated_Friedman_data_v2 if "friedman_correlated" in folder else generate_Friedman_data_v2

    for seed in seeds:

        N, sigma = get_N_sigma(seed)
        dataset_key = f'Friedman_N{N}_p10_sigma{sigma:.2f}_seed{seed}'

        # Build large eval set consistent with training split standardization
        gen_fn = choose_gen(folder)
        X_test, y_test, y_mean, y_std = make_large_eval_set(
            generator_fn=gen_fn,
            N_train=N,
            D=10,
            sigma=sigma,
            seed=seed,
            n_eval=500,
            standardize_y=True
        )

        for model in models:
            try:
                fit = all_fits[dataset_key][model]['posterior']
                W1_samples = fit.stan_variable("W_1")           # (S, P, H)
                W2_samples = fit.stan_variable("W_L")           # (S, H, O)
                b1_samples = fit.stan_variable("hidden_bias")   # (S, O, H)
                b2_samples = fit.stan_variable("output_bias")   # (S, O)
            except KeyError:
                print(f"[SKIP] Model or posterior not found: {dataset_key} -> {model}")
                continue

            S = W1_samples.shape[0]
            rmses = np.zeros(S)
            #print(y_test.shape)
            y_hats = np.zeros((S, y_test.shape[0]))

            for i in range(S):
                W1 = W1_samples[i]
                W2 = W2_samples[i]

                # Apply pruning mask if requested
                if prune_fn is not None and sparsity > 0.0:
                    masks = prune_fn([W1, W2], sparsity)
                    W1 = W1 * masks[0]
                    #W2 = W2 * masks[1]

                y_hat = forward_pass(X_test, W1, b1_samples[i][0], W2, b2_samples[i])
                y_hats[i] = y_hat.squeeze()  # Store the prediction for each sample
                rmses[i] = np.sqrt(np.mean((y_hat.squeeze() - y_test)**2)) * y_std
                
            posterior_mean = np.mean(y_hats, axis=0)
            posterior_mean_rmse = np.sqrt(np.mean((posterior_mean - y_test.squeeze())**2))

            posterior_means.append({
                'seed': seed,
                'N': N,
                'sigma': sigma,
                'model': model,
                'sparsity': sparsity,
                'posterior_mean_rmse': posterior_mean_rmse * y_std
            })

            for i in range(S):
                results.append({
                    'seed': seed,
                    'N': N,
                    'sigma': sigma,
                    'model': model,
                    'sparsity': sparsity,
                    'rmse': rmses[i]
                })

    df_rmse = pd.DataFrame(results)
    df_posterior_rmse = pd.DataFrame(posterior_means)

    return df_rmse, df_posterior_rmse

sparsity_levels = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95]

seeds = [1, 2, 11]
seeds_correlated = [1, 6, 11]

def get_N_sigma(seed):
    if seed == 1:
        N=100
    elif seed == 2:
        N=200
    else:
        N=500
    sigma=1.00
    return N, sigma

def get_N_sigma_correlated(seed):
    if seed == 1:
        N=100
    elif seed == 6:
        N=200
    else:
        N=500
    sigma=1.00
    return N, sigma

In [14]:
df_rmse_tanh, df_posterior_rmse_tanh = {}, {}
df_rmse_tanh_nodewise, df_posterior_rmse_tanh_nodewise = {}, {}
df_rmse_tanh_correlated, df_posterior_rmse_tanh_correlated = {}, {}
df_rmse_tanh_correlated_nodewise, df_posterior_rmse_tanh_correlated_nodewise = {}, {}

for sparsity in sparsity_levels:
    df_rmse_tanh[sparsity], df_posterior_rmse_tanh[sparsity] = compute_sparse_rmse_results(
        seeds, model_names_tanh, tanh_fits, get_N_sigma, forward_pass_tanh, folder = "friedman",
        sparsity=sparsity, prune_fn=local_prune_weights
    )
    
    df_rmse_tanh_nodewise[sparsity], df_posterior_rmse_tanh_nodewise[sparsity] = compute_sparse_rmse_results(
        seeds, model_names_tanh_nodewise, tanh_fits_nodewise, get_N_sigma, forward_pass_tanh, folder = "friedman",
        sparsity=sparsity, prune_fn=local_prune_weights
    )
    
    df_rmse_tanh_correlated[sparsity], df_posterior_rmse_tanh_correlated[sparsity] = compute_sparse_rmse_results(
        seeds_correlated, model_names_tanh_correlated, tanh_fits_correlated, get_N_sigma_correlated, forward_pass_tanh, folder = "friedman_correlated",
        sparsity=sparsity, prune_fn=local_prune_weights
    )
    
    df_rmse_tanh_correlated_nodewise[sparsity], df_posterior_rmse_tanh_correlated_nodewise[sparsity] = compute_sparse_rmse_results(
        seeds_correlated, model_names_tanh_correlated_nodewise, tanh_fits_correlated_nodewise, get_N_sigma_correlated, forward_pass_tanh, folder = "friedman_correlated",
        sparsity=sparsity, prune_fn=local_prune_weights
    )

In [15]:
# import pandas as pd

# df_rmse_full_tanh = pd.concat(
#     [df.assign(sparsity=sparsity) for sparsity, df in df_rmse_tanh.items()],
#     ignore_index=True
# )
# df_rmse_full_tanh_nodewise = pd.concat(
#     [df.assign(sparsity=sparsity) for sparsity, df in df_rmse_tanh_nodewise.items()],
#     ignore_index=True
# )

# df_rmse_full_tanh_correlated = pd.concat(
#     [df.assign(sparsity=sparsity) for sparsity, df in df_rmse_tanh_correlated.items()],
#     ignore_index=True
# )
# df_rmse_full_tanh_correlated_nodewise = pd.concat(
#     [df.assign(sparsity=sparsity) for sparsity, df in df_rmse_tanh_correlated_nodewise.items()],
#     ignore_index=True
# )

import pandas as pd

df_post_tanh_full = pd.concat(
    [df.assign(sparsity=sparsity) for sparsity, df in df_posterior_rmse_tanh.items()],
    ignore_index=True
)
df_post_tanh_full_nodewise = pd.concat(
    [df.assign(sparsity=sparsity) for sparsity, df in df_posterior_rmse_tanh_nodewise.items()],
    ignore_index=True
)

df_post_tanh_full_corr = pd.concat(
    [df.assign(sparsity=sparsity) for sparsity, df in df_posterior_rmse_tanh_correlated.items()],
    ignore_index=True
)
df_post_tanh_full_corr_nodewise = pd.concat(
    [df.assign(sparsity=sparsity) for sparsity, df in df_posterior_rmse_tanh_correlated_nodewise.items()],
    ignore_index=True
)

In [16]:
df_all = pd.concat([df_post_tanh_full, df_post_tanh_full_nodewise])

df_all_corr = pd.concat([df_post_tanh_full_corr, df_post_tanh_full_corr_nodewise])

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

colors = {
    "Gaussian tanh": "C0",
    "Regularized Horseshoe tanh": "C1",
    "Dirichlet Horseshoe tanh": "C2",
    "Dirichlet Horseshoe tanh nodewise": "C2",
    "Dirichlet Student T tanh": "C3",
    "Dirichlet Student T tanh nodewise": "C3",
    "Beta Horseshoe tanh": "C4",
    "Beta Horseshoe tanh nodewise": "C4",
    "Beta Student T tanh": "C5",
    "Beta Student T tanh nodewise": "C5",                 
}

sparsity_levels = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95]

rmse_col = "posterior_mean_rmse"

import matplotlib.pyplot as plt
import pandas as pd

Ns = [100, 200, 500]
dfs = [(df_all, "Uncorrelated"), (df_all_corr, "Correlated")]

fig, axes = plt.subplots(2, 3, figsize=(14, 7), sharex=True, sharey="row")

for r, (df_src, row_title) in enumerate(dfs):
    for c, N in enumerate(Ns):
        ax = axes[r, c]
        df = df_src[(df_src["N"] == N) & (df_src["sparsity"].isin(sparsity_levels))].copy()
        df["sparsity"] = pd.Categorical(df["sparsity"], categories=sparsity_levels, ordered=True)

        for model, g in df.groupby("model", sort=False):
            # put your include/exclude logic here
            if model in ["Gaussian tanh", "Dirichlet Student T tanh", "Beta Student T tanh",
                         "Dirichlet Student T tanh nodewise", "Beta Student T tanh nodewise"]:
            # if model in ["Gaussian tanh", "Regularized Horseshoe tanh", "Beta Student T tanh", "Beta Horseshoe tanh",
            #              "Beta Horseshoe tanh nodewise", "Beta Student T tanh nodewise"]:
                continue

            g = g.sort_values("sparsity")
            lw, alpha, ls, mk = (1.3, 0.7, "--", "v") if "nodewise" in model else (2.2, 1.0, "-", "o")
            ax.plot(g["sparsity"], g[rmse_col], lw=lw, alpha=alpha, ls=ls, marker=mk,
                    color=colors[model], label=model)

        ax.set_title(f"N={N}" if r == 0 else "")
        ax.grid(True, linestyle="--", linewidth=0.5)
        ax.set_xticks(sparsity_levels[: -1])

        if c == 0:
            ax.set_ylabel(f"{row_title}\nRMSE")
        if r == 1:
            ax.set_xlabel("sparsity")

# one legend for the whole figure (deduped)
handles, labels = axes[0,0].get_legend_handles_labels()
by_label = dict(zip(labels, handles))
axes[1, 1].legend(by_label.values(), by_label.keys(),
            loc="upper left",
            #bbox_to_anchor=(1.02, 0.5),
            frameon=False)
axes[0, 1].legend(by_label.values(), by_label.keys(),
            loc="upper left",
            #bbox_to_anchor=(1.02, 0.5),
            frameon=False)

plt.tight_layout(rect=[0, 0, 0.85, 1])
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

colors = {
    "Gaussian tanh": "C0",
    "Regularized Horseshoe tanh": "C1",
    "Dirichlet Horseshoe tanh": "C2",
    "Dirichlet Horseshoe tanh nodewise": "C2",
    "Dirichlet Student T tanh": "C3",
    "Dirichlet Student T tanh nodewise": "C3",
    "Beta Horseshoe tanh": "C4",
    "Beta Horseshoe tanh nodewise": "C4",
    "Beta Student T tanh": "C5",
    "Beta Student T tanh nodewise": "C5",                 
}

sparsity_levels = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95]

rmse_col = "posterior_mean_rmse"

import matplotlib.pyplot as plt
import pandas as pd

Ns = [100, 200, 500]
dfs = [(df_all, "Uncorrelated"), (df_all_corr, "Correlated")]

fig, axes = plt.subplots(2, 3, figsize=(14, 7), sharex=True, sharey='row')

for r, (df_src, row_title) in enumerate(dfs):
    for c, N in enumerate(Ns):
        ax = axes[r, c]
        df = df_src[(df_src["N"] == N) & (df_src["sparsity"].isin(sparsity_levels))].copy()
        df["sparsity"] = pd.Categorical(df["sparsity"], categories=sparsity_levels, ordered=True)

        for model, g in df.groupby("model", sort=False):
            # put your include/exclude logic here
            if model in ["Gaussian tanh", "Dirichlet Horseshoe tanh", "Beta Horseshoe tanh",
                         "Beta Horseshoe tanh nodewise", "Dirichlet Horseshoe tanh nodewise"]:
                continue

            g = g.sort_values("sparsity")
            lw, alpha, ls, mk = (1.3, 0.7, "--", "v") if "nodewise" in model else (2.2, 1.0, "-", "o")
            ax.plot(g["sparsity"], g[rmse_col], lw=lw, alpha=alpha, ls=ls, marker=mk,
                    color=colors[model], label=model)

        ax.set_title(f"N={N}" if r == 0 else "")
        ax.grid(True, linestyle="--", linewidth=0.5)
        ax.set_xticks(sparsity_levels[:-1])

        if c == 0:
            ax.set_ylabel(f"{row_title}\nRMSE")
        if r == 1:
            ax.set_xlabel("sparsity")

# one legend for the whole figure (deduped)
handles, labels = axes[0,0].get_legend_handles_labels()
by_label = dict(zip(labels, handles))
axes[1, 1].legend(by_label.values(), by_label.keys(),
            loc="upper left",
            #bbox_to_anchor=(1.02, 0.5),
            frameon=False)
axes[0, 1].legend(by_label.values(), by_label.keys(),
            loc="upper left",
            #bbox_to_anchor=(1.02, 0.5),
            frameon=False)

#plt.ylim((0.1, 2))
plt.tight_layout(rect=[0, 0, 0.85, 1])
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

colors = {
    "Gaussian tanh": "C0",
    "Regularized Horseshoe tanh": "C1",
    "Dirichlet Horseshoe tanh": "C2",
    "Dirichlet Horseshoe tanh nodewise": "C2",
    "Dirichlet Student T tanh": "C3",
    "Dirichlet Student T tanh nodewise": "C3",
    "Beta Horseshoe tanh": "C4",
    "Beta Horseshoe tanh nodewise": "C4",
    "Beta Student T tanh": "C5",
    "Beta Student T tanh nodewise": "C5",                 
}

sparsity_levels = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95]

rmse_col = "posterior_mean_rmse"

import matplotlib.pyplot as plt
import pandas as pd

Ns = [100, 200, 500]
dfs = [(df_all, "Uncorrelated"), (df_all_corr, "Correlated")]

fig, axes = plt.subplots(2, 3, figsize=(14, 7), sharex=True, sharey=True)

for r, (df_src, row_title) in enumerate(dfs):
    for c, N in enumerate(Ns):
        ax = axes[r, c]
        df = df_src[(df_src["N"] == N) & (df_src["sparsity"].isin(sparsity_levels))].copy()
        df["sparsity"] = pd.Categorical(df["sparsity"], categories=sparsity_levels, ordered=True)

        for model, g in df.groupby("model", sort=False):
            # put your include/exclude logic heres
            if model in ["Gaussian tanh", "Beta Student T tanh", "Beta Horseshoe tanh",
                         "Beta Horseshoe tanh nodewise", "Beta Student T tanh nodewise", ]:
                continue

            g = g.sort_values("sparsity")
            lw, alpha, ls, mk = (1.3, 0.7, "--", "v") if "nodewise" in model else (2.2, 1.0, "-", "o")
            ax.plot(g["sparsity"], g[rmse_col], lw=lw, alpha=alpha, ls=ls, marker=mk,
                    color=colors[model], label=model)

        ax.set_title(f"N={N}" if r == 0 else "")
        ax.grid(True, linestyle="--", linewidth=0.5)
        ax.set_xticks(sparsity_levels[:-1])

        if c == 0:
            ax.set_ylabel(f"{row_title}\nRMSE")
        if r == 1:
            ax.set_xlabel("sparsity")

# one legend for the whole figure (deduped)
handles, labels = axes[0,0].get_legend_handles_labels()
by_label = dict(zip(labels, handles))
axes[1, 1].legend(by_label.values(), by_label.keys(),
            loc="upper left",
            #bbox_to_anchor=(1.02, 0.5),
            frameon=False)
axes[0, 1].legend(by_label.values(), by_label.keys(),
            loc="upper left",
            #bbox_to_anchor=(1.02, 0.5),
            frameon=False)

#plt.ylim((0.1, 2))
plt.tight_layout(rect=[0, 0, 0.85, 1])
plt.show()
